<a href="https://colab.research.google.com/github/gpanagioto/projects_socialgraphs22/blob/main/sg22_assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Formalia:
Please read the [assignment overview page](https://github.com/SocialComplexityLab/socialgraphs2022/wiki/Assignments) carefully before proceeding. This page contains information about formatting (including formats etc), group sizes, and many other aspects of handing in the assignment.

If you fail to follow these simple instructions, it will negatively impact your grade!

Due date and time: The assignment is due on Tuesday November 1st, 2022 at 23:55. Hand in your IPython notebook file (with extension .ipynb) via http://peergrade.io/ (we won't be doing peergrading, but we'll still use http://peergrade.io/ for the handin.)

----

# Assignment 2: Network Science

_Course: 02805 - **Social Graphs and Interactions**_ <br>
_Course responsible: **Sune Lehmann Jørgensen**_ <br>
_DTU - **Technical University of Denmark**_  
_Due date - **01/11/2022**_ <br>
_Students - **Nikos Karageorgos, John Manganas, Georgios Panagiotopoulos**_

---

## Table of Contents:
- [__Part 0: Data__](#0.)

- [__Part 1: Basic Stats__](#1.)

- [__Part 2: Communities__](#2.)

- [__Part 3: Sentiment__](#3.)

---

Installation and Importing of all the necessary libraries we will need

In [17]:
%%capture
!pip install wordcloud
from io import BytesIO
import requests
from scipy import stats as st
from wordcloud import WordCloud, STOPWORDS
from nltk import *
import pprint
from nltk import word_tokenize
#nltk.download('punkt')
import pandas as pd
from sqlalchemy import create_engine
import string
import re
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx
import pickle
import numpy as np
import community.community_louvain as community_louvain

Our DC and Marvel superhero dataset after the proper extraction have been stored in a cloud PostgreSQL Database, easily and quickly accessible.

In [12]:
# Defining function for importing data from the cloud DB. The function returns two dataframes, one for each universe with two columns character_name, wiki_text
def DataImport(table_name):
  host="ec2-54-75-184-144.eu-west-1.compute.amazonaws.com"
  port="5432" 
  dbname="dab1kopm5t3l06"
  user="kpervzhazofybh" 
  password="0d1b5470c51c481880eed267865a8529bdc671f8cb90702d6dcb9e7c199d02ee"

  Engine   = create_engine('postgresql+psycopg2://{}:{}@{}:{}/{}'.format(user,password,host,port,dbname))

  # Connect to PostgreSQL server
  dbConnection = Engine.connect();

  # Read data from PostgreSQL database table and load into a DataFrame instance
  df = pd.read_sql("select * from {}".format(table_name), dbConnection)
  pd.set_option('display.expand_frame_repr', False)

  return df

In [ ]:
# Our Network has been stored as a pickle file in our GitHub Repository of this second assignment
mLink = 'https://github.com/gpanagioto/projects_socialgraphs22/blob/main/Assignment2/SuperHeroesGraph.gpickle?raw=true'
mfile = BytesIO(requests.get(mLink).content)
G = pickle.load(mfile)

#G_dir = pickle.load(open('/content/drive/MyDrive/DTU/02805 Social graphs and interactions/SuperHeroesGraph.gpickle', 'rb'))

<a id='0.'></a>
# Part 0: Data 


* Write a short paragraph describing the network. The paragraph should contain the following information

  * The number of nodes and links.
  * The average, median, mode, minimum and maximum value of the network's in-degree.s And of the out-degrees.

In [22]:
def Measures(Graph, TypeOfDegree, Data):

  if TypeOfDegree == 'In':
    values = list(dict(Graph.in_degree(Data)).values())
  else:
    values = list(dict(Graph.out_degree(Data)).values())
  
  mean = np.mean(values)
  print("The average value is {}.".format(mean))
  median = np.median(values)
  print("\nThe median is {}.".format(median))
  mode = st.mode(values)
  print("\nThe mode is {}.".format(list(mode)[0][0]))
  min = np.min(values)
  print("\nThe min is {}.".format(min))
  max = np.max(values)
  print("\nThe max is {}.".format(max))

  return mean, median, list(mode)[0][0], min, max  

In [23]:
print('The number of nodes of the network are {}. The number of edges of the network are {}.'.format(G.number_of_nodes(),G.number_of_edges()))

The number of nodes of the network are 2538. The number of edges of the network are 29998.


In [24]:
Data = list(G.nodes())
TypeOfDegree = 'In'
average, median, mode, min, max = Measures(G, TypeOfDegree, Data)

The average value is 11.819542947202521.

The median is 4.0.

The mode is 1.

The min is 0.

The max is 448.


In [25]:
Data = list(G.nodes())
TypeOfDegree = 'Out'
average, median, mode, min, max = Measures(G, TypeOfDegree, Data)

The average value is 11.819542947202521.

The median is 7.0.

The mode is 0.

The min is 0.

The max is 112.


* We also want the degree distributions and a plot of the network

  * Create in- and out-going degree distributions as described in Lecture 5.
  * Estimate the slope of the incoming degree distribtion as described in Lecture 5.
  * Plot the network using the Force Atlas algorithm as described in Lecture 5.
